In [1]:
## basic

    mkdir ansible
    cd ansible/
    source ~/venvs/ansible/bin/activate
    pip install wheel ansible
    mkdir install
    cd install/
    vim inventory

        [all]
        127.0.0.1 

        [all:vars]
        ansible_port=9922
        ansible_user=root
        ask_pass=true

    ansible all -i inventory --user root --ask-pass -m command -a id

    ansible all -i inventory --user root --ask-pass -m command -a id -o

    ansible all -i inventory -m shell -a env

    touch ansbile.cfg

    ansible all -i inventory --ask-pass -m copy -a 'content="Managed by Ansible\n" dest=/etc/motd'

In [2]:
## ansible doc

    ansible-doc -l
    ansible-doc apt
    ansible-doc -s apt

In [3]:
## working with modules

    ansible -m ping all
    
    ansible all -i inventory --ask-pass -m ping
    
    vim playbook.yaml
    
        tasks:
        - name: Install a package
          yum:
              name: vsftpd
              state: latest

In [4]:
## creating yaml files

    vim mycode.yml
    
        ---
        - apple
        -bird
        ...
        
    ansible-playbook --syntax-check mycode.yml 
    
    pip install ansible-lint
    
    ansible-lint mycode.yml 

In [5]:
## creating playbook

    vim simple.yml

        ---
        # My first playbook
        - name: simple playbook
          hosts: 127.0.0.1
          user: root
          become: yes
          become_method: sudo
          become_user: root
          tasks:
          - name: install vsftpd
            yum: name=vsftpd
          - name: enable vsftpd
            service: name=vsftpd enabled=true
          - name: create readme file
            copy:
              content: "welcome to my ftp server"
              dest: /var/ftp/pub/README
              force: no
              mode: 0444
        ...

    ansible-playbook -i inventory --ask-pass simple.yml

In [7]:
## running playbook

normal

    ansible-playbook -i inventory --ask-pass simple.yml

dry run

    ansible-playbook -i inventory --ask-pass -C simple.yml

step

    ansible-playbook -i inventory --ask-pass --step simple.yml 

In [9]:
## group variables

    mkdir vardemo
    cd vardemo
    vim inventory

        [webservers]
        server1.example.com
        server2.example.com

        [ftpservers]
        server3.example.com
        server4.example.com

    mkdir group_vars
    cd group_vars
    vim webservers

        package: httpd

    vim ftpservers

        package: vsftpd



In [10]:
## facts

    ansible all -i inventory --user root --ask-pass -m setup

    ansible all -i inventory --user root --ask-pass -m setup -a 'filter=ansible_kernel'

In [11]:
## inclusions

    mkdir inclusions
    cd inclusions
    vim setup.yml

        ---
        - name: Deploy services
          hosts: all
          tasks:
          - name: Read the tasks.yml file to find what to do
            include: tasks.yml
            vars:
              package: samba
              service: smb
              state: started
            register: output

          - name: debugs the included tasks
            debug:
              var: output
        ...

    vim tasks.yml

        - name: install the {{ package }} package
          yum:
            name: "{{ package }}"
            state: latest

        - name: start the {{ service }} package
          service:
            name: "{{ service }}"
            state: "{{ state }}"

    vim packages.yml

        ---
        packages:
          my_pkg: httpd

    vim installpkg.yml

        ---
        - name: install some packages
          hosts: all
          tasks:
            - name: include packages
              include_vars: packages.yml

            - name: installs {{ packages.my_pkg }}
              yum:
                name: "{{ packages.my_pkg }}"
                state: latest

    vim inventory

        [all]
        127.0.0.1

        [all:vars]
        ansible_port=9922
        ansible_user=root
        
    ansible-playbook -i inventory --ask-pass installpkg.yml
    
    ansible-playbook -i inventory --ask-pass installpkg.yml -e "packages.my_pkg=vsftpd"

In [12]:
## lab 4
### will able to run on centos and ubuntu due to custom facts

    mkdir lab4
    cd lab4
    vim inventory

        [all]
        managed1.ansible.local
        managed2.ansible.local
        control.ansible.local

        [lamp]
        managed2.ansible.local

        [file]
        managed2.ansible.local

    vim custom.fact

        [packages]
        smb_package = samba
        ftp_package = vsftpd
        db_package = mariadb-server
        web_package = httpd

        [services]
        smb_service = smb
        ftp_service = vsftpd
        db_service = mariadb
        web_service = httpd

    vim copy-facts.yml

        ---
        - name: Install remote facts
          hosts: all
          vars:
            remote_dir: /etc/ansible/facts.d
            facts_file: custom.fact
          tasks:
            - name: create remote directory
              file:
                state: directory
                recurse: yes
                path: "{{ remote_dir }}"
            - name: install new facts
              copy:
                src: "{{ facts_file }}"
                dest: "{{ remote_dir }}"

    ansible-playbook -i inventory copy-facts.yml

    ansible -m setup -a 'filter=ansible_local' -i inventory all

    mkdir vars
    cd vars
    vim allvars.yml

        ---
        web_root: /var/www/html
        ftp_root: /var/ftp

    cd ..
    mkdir tasks
    cd tasks
    vim file.yml

        ---
        - name: install and start the file servers
          yum:
            name:
              - "{{ ansible_local.custom.packages.smb_package }}"
              - "{{ ansible_local.custom.packages.ftp_package }}"
            state: latest

        - name: start samba server
          service:
            name: "{{ ansible_local.custom.packages.smb_service }}"
            state: started
            enabled: true

        - name: start the ftp service
          service:
            name: "{{ ansible_local.custom.packages.ftp_service }}"
            state: started
            enabled: true

    vim lamp.yml

        ---
        - name: install and start the servers
          yum:
            name:
              - "{{ ansible_local.custom.packages.db_package }}"
              - "{{ ansible_local.custom.packages.web_package }}"
            state: latest

        - name: start database server
          service:
            name: "{{ ansible_local.custom.packages.db_service }}"
            state: started
            enabled: true

        - name: start the web service
          service:
            name: "{{ ansible_local.custom.packages.web_service }}"
            state: started
            enabled: true

    cd ..
    vim playbook.yml

        ---
        - hosts: all
          vars:
            firewall: firewalld

          tasks:
            - name: install the firewall
              yum:
                name: "{{ firewall }}"
                state: latest

            - name: start the firewall
              service:
                name: "{{ firewall }}"
                state: started
                enabled: true

        - hosts: lamp
          tasks:
            - name: include the variable file
              include_vars: vars/allvars.yml

            - name: include the tasks
              include: tasks/lamp.yml

            - name: open the port for the web server
              firewalld:
                service: http
                state: enabled
                immediate: true
                permanent: true

            - name: create index.html
              copy:
                content: "{{ ansible_fqdn }}({{ ansible_default_ipv4.address }}) managed by Ansible\n"
                dest: "{{ web_root }}/index.html"

        - hosts: file
          tasks:
            - name: include the variable file
              include_vars: vars/allvars.yml

            - name: include the tasks
              include: tasks/file.yml

            - name: open the port for the ftp service
              firewalld:
                service: samba
                immediate: true
                permanent: true

            - name: open the port for the smb service
              firewalld:
                service: ftp
                state: enabled
                immediate: true
                permanent: true


    ansible-playbook playbook.yml -i inventory



In [14]:
## flow control with loop and nested_loop

    vim createusers.yml

        - name: create users
          hosts: all
          tasks:
            - user:
                name: "{{ item.name }}"
                state: present
                groups: "{{ item.groups }}"
              with_items:
                - { name: 'linda', groups: 'wheel' }
                - { name: 'lisa', groups: 'root' }
                
    
    vim makedbusers.yml
    
        - name: give users access to multiple databases
          mysql_user:
            name: "{{ item[0] }}"
            priv: "{{ item[1] }}.*:ALL"
            append_privs: yes
            password: "foo"
          with_nested:
            - ['linda', 'lisa']
            - ['clientdb', 'employeedb', 'providerdb']    

In [15]:
## working with conditionals

    ansible all -i inventory --user root --ask-pass -m setup -a 'filter=ansible_mounts'

    vim installif.yml

        ---
        - hosts: all
          tasks:
            - name: install package if sufficient disk space
              yum:
                name: mariadb-server
                state: latest
              with_items: "{{ ansible_mounts }}"
              when: item.mount == "/" and item.size_available > 200000000

    ansible-playbook -i inventory --ask-pass installif.yml


    vim installif2.yml

        ---
        - hosts: all
          tasks:
            - name: check mariadb status
              command: /usr/bin/systemctl is-active mariadb
              ignore_errors: yes
              register: result

            - name: install http if mariadb is active
              yum:
                name: httpd
                state: installed
              when: result.rc == 0

            - name: start httpd
              service:
                name: httpd
                state: started
                
                

    ansible-playbook -i inventory --ask-pass installif2.yml 
    


    vim setup_controller.yml

        ---
        - hosts: all
          tasks:
            - name: install packstack
              yum:
                name: openstack-packstack
                state: latest
              register: result

            - name: generate answers file if packstack is installed
              command:
                packstack --gen-answer-file /root/answers.txt
              when: result.rc == 0

In [16]:
## jinja templates

    vim motd.j2

        This is the system {{ ansible_hostname }}.
        Today it is {{ ansible_date_time.dat }}.
        Only use this system is {{ system_owner }} has granted you permission


    vim motd.yml

        ---
          - hosts: all
            user: root
            become: true
            vars:
              system_owner: anna@example.com
            tasks:
              - template:
                  src: motd.j2
                  dest: /etc/motd
                  owner: root
                  group: root
                  mode: 0644
